<a href="https://colab.research.google.com/github/jrebull/AnaliticaPrescriptiva/blob/main/sudoku_solver_con_pyomo_y_linearexpression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  sudoku-solver-con-pyomo-y-linearexpression

Este notebook resuelve cualquier puzzle de Sudoku utilizando **Programación Lineal Entera** con la librería **Pyomo** en Python.

## Características Principales

- **Modelo Matemático**: El Sudoku se modela como un problema de satisfacción de restricciones, donde cada celda es una variable de decisión.
- **Optimización de Rendimiento**: Se utiliza `LinearExpression` para construir las restricciones del modelo. Este método es significativamente más rápido que la sobrecarga de operadores tradicional de Pyomo (`sum()`), especialmente para modelos grandes, ya que evita la creación de múltiples objetos intermedios.
- **Comparación**: El notebook incluye una comparación de rendimiento directa entre el método optimizado y el método tradicional para demostrar la ganancia en velocidad.

**Autor:** Alex / Angel / Javier / Nohemi   
**Fecha:** 2025

Celda 2: Instalación e Importaciones (Código)
Esta primera celda de código instala las dependencias necesarias y luego importa todas las librerías que usaremos.

In [1]:
# ===================================================================
# PASO 1: INSTALACIÓN DE LIBRERÍAS Y SOLVER
# ===================================================================
# Usamos "!" para ejecutar comandos de la terminal desde la celda.
# -q es para que la instalación sea "silenciosa".

# Instalamos la librería Pyomo para modelado de optimización
!pip install -q pyomo

# Instalamos un solver (GLPK). Pyomo lo necesita para resolver los modelos.
!apt-get install -y -qq glpk-utils


# ===================================================================
# PASO 2: IMPORTACIÓN DE LIBRERÍAS A PYTHON
# ===================================================================
print("✅ Instalaciones completas. Importando librerías...")

import time
import numpy as np
import pyomo.environ as pyo
from pyomo.common.timing import TicTocTimer, report_timing
from pyomo.core.expr.numeric_expr import LinearExpression

print("👍 ¡Todo listo!")

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 126435 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

Celda 3: Funciones de Configuración y Utilidades (Código)
Aquí definimos funciones auxiliares y creamos el mapeo de las subcuadrículas que será usado por el modelo.

In [2]:
# ===========================
# CONFIGURACIÓN Y UTILIDADES
# ===========================

def crear_mapeo_subcuadriculas():
    """
    Crea un diccionario que mapea cada subcuadrícula a sus celdas correspondientes.
    """
    mapeo = {}
    for k in range(1, 10):
        fila_base = ((k - 1) // 3) * 3
        col_base = ((k - 1) % 3) * 3
        celdas = [(fila_base + i + 1, col_base + j + 1) for i in range(3) for j in range(3)]
        mapeo[k] = celdas
    return mapeo

# Mapeo global de subcuadrículas, disponible para todas las funciones
MAPEO_SUBCUADRICULAS = crear_mapeo_subcuadriculas()

def imprimir_sudoku(matriz, titulo="Sudoku"):
    """
    Imprime un Sudoku de forma legible en la consola.
    """
    print(f"\n{titulo}")
    print("╔═══════╤═══════╤═══════╗")

    # Convertir lista de tuplas a matriz si es necesario
    if not isinstance(matriz, np.ndarray):
        temp = np.zeros((9, 9), dtype=int)
        for (f, c, v) in matriz:
            temp[f-1][c-1] = v
        matriz = temp

    for i in range(9):
        if i > 0 and i % 3 == 0:
            print("╟───────┼───────┼───────╢")
        fila = "║"
        for j in range(9):
            if j > 0 and j % 3 == 0:
                fila += "│"
            valor = matriz[i][j]
            fila += f" {valor} " if valor != 0 else " · "
        fila += "║"
        print(fila)

    print("╚═══════╧═══════╧═══════╝")

print("✓ Funciones de utilidad cargadas.")

✓ Funciones de utilidad cargadas.


Celda 4: Definición del Modelo Optimizado (Código)
Esta es la celda principal donde se construye el modelo de Pyomo usando la técnica optimizada con LinearExpression.

In [3]:
# ==========================================
# MODELO OPTIMIZADO CON LINEAR EXPRESSION
# ==========================================

def crear_modelo_sudoku_optimizado(tablero_inicial):
    """
    Crea un modelo de Sudoku optimizado usando LinearExpression para todas las restricciones.
    """
    modelo = pyo.ConcreteModel(name="Sudoku_Optimizado")

    # ===== CONJUNTOS =====
    modelo.FILAS = pyo.RangeSet(1, 9)
    modelo.COLUMNAS = pyo.RangeSet(1, 9)
    modelo.VALORES = pyo.RangeSet(1, 9)
    modelo.SUBCUADRICULAS = pyo.RangeSet(1, 9)

    # ===== VARIABLES DE DECISIÓN =====
    modelo.y = pyo.Var(modelo.FILAS, modelo.COLUMNAS, modelo.VALORES, within=pyo.Binary)

    # ===== FIJAR VALORES INICIALES (PISTAS) =====
    for (fila, col, valor) in tablero_inicial:
        modelo.y[fila, col, valor].fix(1)
        for v in modelo.VALORES:
            if v != valor:
                modelo.y[fila, col, v].fix(0)

    # ===== FUNCIÓN OBJETIVO (TRIVIAL) =====
    modelo.objetivo = pyo.Objective(expr=1.0)

    # ===========================
    # RESTRICCIONES CON LINEAR EXPRESSION
    # ===========================

    # 1. Exactamente un valor por celda
    def restriccion_unicidad_celda(modelo, f, c):
        variables = [modelo.y[f, c, v] for v in modelo.VALORES]
        return LinearExpression(linear_vars=variables, linear_coefs=[1.0]*9) == 1
    modelo.unicidad_celda = pyo.Constraint(modelo.FILAS, modelo.COLUMNAS, rule=restriccion_unicidad_celda)

    # 2. Cada valor aparece una vez por fila
    def restriccion_fila(modelo, f, v):
        variables = [modelo.y[f, c, v] for c in modelo.COLUMNAS]
        return LinearExpression(linear_vars=variables, linear_coefs=[1.0]*9) == 1
    modelo.restriccion_fila = pyo.Constraint(modelo.FILAS, modelo.VALORES, rule=restriccion_fila)

    # 3. Cada valor aparece una vez por columna
    def restriccion_columna(modelo, c, v):
        variables = [modelo.y[f, c, v] for f in modelo.FILAS]
        return LinearExpression(linear_vars=variables, linear_coefs=[1.0]*9) == 1
    modelo.restriccion_columna = pyo.Constraint(modelo.COLUMNAS, modelo.VALORES, rule=restriccion_columna)

    # 4. Cada valor aparece una vez por subcuadrícula
    def restriccion_subcuadricula(modelo, s, v):
        celdas = MAPEO_SUBCUADRICULAS[s]
        variables = [modelo.y[f, c, v] for (f, c) in celdas]
        return LinearExpression(linear_vars=variables, linear_coefs=[1.0]*9) == 1
    modelo.restriccion_subcuadricula = pyo.Constraint(modelo.SUBCUADRICULAS, modelo.VALORES, rule=restriccion_subcuadricula)

    return modelo

print("✓ Función para crear el modelo optimizado cargada.")

✓ Función para crear el modelo optimizado cargada.


Celda 5: Funciones de Solución y Extracción (Código)
Estas funciones toman el modelo, lo envían al solver y procesan los resultados.

In [4]:
# ===========================
# FUNCIONES DE SOLUCIÓN
# ===========================

def resolver_sudoku(tablero_inicial, modelo_func, solver='glpk', mostrar_tiempos=True):
    """
    Función genérica para resolver un Sudoku dado un tablero y una función de creación de modelo.
    """
    tiempo_inicio_total = time.time()

    # Construir modelo
    print(f"\nConstruyendo modelo ({modelo_func.__name__})...")
    tiempo_inicio_construccion = time.time()
    modelo = modelo_func(tablero_inicial)
    tiempo_construccion = time.time() - tiempo_inicio_construccion

    # Resolver modelo
    print(f"Resolviendo con {solver}...")
    opt = pyo.SolverFactory(solver)
    tiempo_inicio_solucion = time.time()
    resultado = opt.solve(modelo, tee=False)
    tiempo_solucion = time.time() - tiempo_inicio_solucion

    tiempo_total = time.time() - tiempo_inicio_total

    # Verificar y extraer solución
    if resultado.solver.termination_condition != pyo.TerminationCondition.optimal:
        print("\n¡No se encontró solución óptima!")
        return None, tiempo_construccion, tiempo_solucion, tiempo_total

    solucion = np.zeros((9, 9), dtype=int)
    for f in modelo.FILAS:
        for c in modelo.COLUMNAS:
            for v in modelo.VALORES:
                if pyo.value(modelo.y[f, c, v]) >= 0.5:
                    solucion[f-1][c-1] = v
                    break

    if mostrar_tiempos:
        print("\n" + "="*40)
        print("ESTADÍSTICAS DE RENDIMIENTO")
        print(f"Tiempo de construcción: {tiempo_construccion:.4f} s")
        print(f"Tiempo de solución: {tiempo_solucion:.4f} s")
        print(f"Tiempo total: {tiempo_total:.4f} s")
        print("="*40)

    return solucion, tiempo_construccion, tiempo_solucion, tiempo_total

print("✓ Función para resolver y extraer la solución cargada.")

✓ Función para resolver y extraer la solución cargada.


Celda 6: Ejecución Principal (Resolver un Sudoku) (Código)
¡Aquí es donde ocurre la magia! Definimos un puzzle y lo resolvemos.

In [5]:
# ===========================
# EJECUCIÓN DEL SOLUCIONADOR
# ===========================

# Puzzle de ejemplo (considerado difícil)
sudoku_dificil = [
    (1,1,5), (1,2,3), (1,5,7),
    (2,1,6), (2,4,1), (2,5,9), (2,6,5),
    (3,2,9), (3,3,8), (3,8,6),
    (4,1,8), (4,5,6), (4,9,3),
    (5,1,4), (5,4,8), (5,6,3), (5,9,1),
    (6,1,7), (6,5,2), (6,9,6),
    (7,2,6), (7,7,2), (7,8,8),
    (8,4,4), (8,5,1), (8,6,9), (8,9,5),
    (9,5,8), (9,8,7), (9,9,9)
]

print("### RESOLVIENDO PUZZLE DIFÍCIL CON MÉTODO OPTIMIZADO ###")
imprimir_sudoku(sudoku_dificil, "PUZZLE INICIAL")

# Resolver usando la función de modelo optimizado
solucion_opt, _, _, _ = resolver_sudoku(sudoku_dificil, crear_modelo_sudoku_optimizado)

if solucion_opt is not None:
    imprimir_sudoku(solucion_opt, "SOLUCIÓN ENCONTRADA")

### RESOLVIENDO PUZZLE DIFÍCIL CON MÉTODO OPTIMIZADO ###

PUZZLE INICIAL
╔═══════╤═══════╤═══════╗
║ 5  3  · │ ·  7  · │ ·  ·  · ║
║ 6  ·  · │ 1  9  5 │ ·  ·  · ║
║ ·  9  8 │ ·  ·  · │ ·  6  · ║
╟───────┼───────┼───────╢
║ 8  ·  · │ ·  6  · │ ·  ·  3 ║
║ 4  ·  · │ 8  ·  3 │ ·  ·  1 ║
║ 7  ·  · │ ·  2  · │ ·  ·  6 ║
╟───────┼───────┼───────╢
║ ·  6  · │ ·  ·  · │ 2  8  · ║
║ ·  ·  · │ 4  1  9 │ ·  ·  5 ║
║ ·  ·  · │ ·  8  · │ ·  7  9 ║
╚═══════╧═══════╧═══════╝

Construyendo modelo (crear_modelo_sudoku_optimizado)...
Resolviendo con glpk...

ESTADÍSTICAS DE RENDIMIENTO
Tiempo de construcción: 0.0080 s
Tiempo de solución: 0.0347 s
Tiempo total: 0.0429 s

SOLUCIÓN ENCONTRADA
╔═══════╤═══════╤═══════╗
║ 5  3  4 │ 6  7  8 │ 9  1  2 ║
║ 6  7  2 │ 1  9  5 │ 3  4  8 ║
║ 1  9  8 │ 3  4  2 │ 5  6  7 ║
╟───────┼───────┼───────╢
║ 8  5  9 │ 7  6  1 │ 4  2  3 ║
║ 4  2  6 │ 8  5  3 │ 7  9  1 ║
║ 7  1  3 │ 9  2  4 │ 8  5  6 ║
╟───────┼───────┼───────╢
║ 9  6  1 │ 5  3  7 │ 2  8  4 ║
║ 2  8  7 │ 4  1 

Celda 7: Comparación de Rendimiento (Código)
Esta celda define el modelo tradicional y ejecuta una comparación para demostrar la mejora de rendimiento.

In [6]:
# ====================================
# COMPARACIÓN CON MÉTODO TRADICIONAL
# ====================================

def crear_modelo_tradicional(tablero_inicial):
    """
    Crea un modelo de Sudoku usando sobrecarga de operadores (método tradicional).
    """
    modelo = pyo.ConcreteModel(name="Sudoku_Tradicional")
    modelo.FILAS = pyo.RangeSet(1, 9)
    modelo.COLUMNAS = pyo.RangeSet(1, 9)
    modelo.VALORES = pyo.RangeSet(1, 9)
    modelo.SUBCUADRICULAS = pyo.RangeSet(1, 9)
    modelo.y = pyo.Var(modelo.FILAS, modelo.COLUMNAS, modelo.VALORES, within=pyo.Binary)

    for (f, c, v) in tablero_inicial:
        modelo.y[f, c, v].fix(1)

    modelo.objetivo = pyo.Objective(expr=1.0)

    # Restricciones con sum() de Python
    modelo.unicidad_celda = pyo.Constraint(modelo.FILAS, modelo.COLUMNAS, rule=lambda m, f, c: sum(m.y[f, c, v] for v in m.VALORES) == 1)
    modelo.restriccion_fila = pyo.Constraint(modelo.FILAS, modelo.VALORES, rule=lambda m, f, v: sum(m.y[f, c, v] for c in m.COLUMNAS) == 1)
    modelo.restriccion_columna = pyo.Constraint(modelo.COLUMNAS, modelo.VALORES, rule=lambda m, c, v: sum(m.y[f, c, v] for f in m.FILAS) == 1)
    modelo.restriccion_subcuadricula = pyo.Constraint(modelo.SUBCUADRICULAS, modelo.VALORES, rule=lambda m, s, v: sum(m.y[f, c, v] for (f, c) in MAPEO_SUBCUADRICULAS[s]) == 1)

    return modelo

print("### INICIANDO COMPARACIÓN DE RENDIMIENTO ###")

# Puzzle para la comparación
sudoku_medio = [
    (1,2,4), (1,3,3), (1,5,8), (1,7,2), (1,8,5), (2,1,6), (3,6,1), (3,8,9), (3,9,4),
    (4,1,9), (4,6,4), (4,8,7), (5,4,6), (5,6,8), (6,2,1), (6,4,2), (6,9,3),
    (7,1,8), (7,2,2), (7,4,5), (8,9,5), (9,2,3), (9,3,4), (9,5,9), (9,7,7), (9,8,1)
]
imprimir_sudoku(sudoku_medio, "Puzzle para Comparación")


# Ejecutar y medir ambos métodos
sol_trad, t_const_trad, t_sol_trad, t_total_trad = resolver_sudoku(sudoku_medio, crear_modelo_tradicional, mostrar_tiempos=False)
sol_opt, t_const_opt, t_sol_opt, t_total_opt = resolver_sudoku(sudoku_medio, crear_modelo_sudoku_optimizado, mostrar_tiempos=False)

# Imprimir resultados de la comparación
print("\n" + "="*60)
print("RESULTADOS DE LA COMPARACIÓN")
print("="*60)
print(f"{'MÉTODO':<25} | {'T. CONSTRUCCIÓN':<15} | {'T. TOTAL':<15}")
print("-"*60)
print(f"{'Tradicional (sum)':<25} | {t_const_trad:<15.4f} | {t_total_trad:<15.4f}")
print(f"{'Optimizado (LinearExpr)':<25} | {t_const_opt:<15.4f} | {t_total_opt:<15.4f}")
print("="*60)

mejora = ((t_const_trad - t_const_opt) / t_const_trad) * 100
print(f"🚀 Mejora en tiempo de construcción del modelo: {mejora:.1f}% más rápido.")

### INICIANDO COMPARACIÓN DE RENDIMIENTO ###

Puzzle para Comparación
╔═══════╤═══════╤═══════╗
║ ·  4  3 │ ·  8  · │ 2  5  · ║
║ 6  ·  · │ ·  ·  · │ ·  ·  · ║
║ ·  ·  · │ ·  ·  1 │ ·  9  4 ║
╟───────┼───────┼───────╢
║ 9  ·  · │ ·  ·  4 │ ·  7  · ║
║ ·  ·  · │ 6  ·  8 │ ·  ·  · ║
║ ·  1  · │ 2  ·  · │ ·  ·  3 ║
╟───────┼───────┼───────╢
║ 8  2  · │ 5  ·  · │ ·  ·  · ║
║ ·  ·  · │ ·  ·  · │ ·  ·  5 ║
║ ·  3  4 │ ·  9  · │ 7  1  · ║
╚═══════╧═══════╧═══════╝

Construyendo modelo (crear_modelo_tradicional)...
Resolviendo con glpk...

Construyendo modelo (crear_modelo_sudoku_optimizado)...
Resolviendo con glpk...

RESULTADOS DE LA COMPARACIÓN
MÉTODO                    | T. CONSTRUCCIÓN | T. TOTAL       
------------------------------------------------------------
Tradicional (sum)         | 0.0096          | 0.0545         
Optimizado (LinearExpr)   | 0.0078          | 0.0380         
🚀 Mejora en tiempo de construcción del modelo: 18.7% más rápido.


Celda 8: Verificación de la Solución (Código)
Finalmente, una celda para verificar formalmente que la solución encontrada es válida y cumple todas las reglas del Sudoku.

In [7]:
# ===========================
# VERIFICACIÓN DE LA SOLUCIÓN
# ===========================

def verificar_solucion(solucion):
    """
    Verifica que una solución de Sudoku sea válida.
    """
    es_valida = True
    # Verificar filas y columnas
    for i in range(9):
        if len(set(solucion[i, :])) != 9: es_valida = False; print(f"Error en fila {i+1}")
        if len(set(solucion[:, i])) != 9: es_valida = False; print(f"Error en columna {i+1}")

    # Verificar subcuadrículas
    for i in range(0, 9, 3):
        for j in range(0, 9, 3):
            if len(set(solucion[i:i+3, j:j+3].flatten())) != 9:
                es_valida = False; print(f"Error en subcuadrícula ({i//3+1}, {j//3+1})")

    if es_valida:
        print("\n✓ La solución es VÁLIDA. Todas las restricciones se cumplen.")
    else:
        print("\n✗ La solución NO es válida.")
    return es_valida

# Verificar la solución del puzzle difícil que resolvimos en la celda 6
if 'solucion_opt' in locals() and solucion_opt is not None:
    print("\n### VERIFICANDO LA SOLUCIÓN DEL PUZZLE DIFÍCIL ###")
    verificar_solucion(solucion_opt)


### VERIFICANDO LA SOLUCIÓN DEL PUZZLE DIFÍCIL ###

✓ La solución es VÁLIDA. Todas las restricciones se cumplen.
